In [35]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
 
inception_model = 'D:/python_code/tensorflow_inception_graph/tensorflow_inception_graph.pb'
print (1)
# 加载inception模型
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
 
X = tf.placeholder(np.float32, name='input')
with tf.gfile.FastGFile(inception_model, 'rb') as f:
	graph_def = tf.GraphDef()
	graph_def.ParseFromString(f.read())
imagenet_mean = 117.0
preprocessed = tf.expand_dims(X-imagenet_mean, 0)
tf.import_graph_def(graph_def, {'input':preprocessed})
 
layers = [op.name for op in graph.get_operations() if op.type=='Conv2D' and 'import/' in op.name]
feature_nums = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]
 
print('layers:', len(layers))   # 59
print('feature:', sum(feature_nums))  # 7548
 
# deep dream
def deep_dream(obj, img_noise=np.random.uniform(size=(224,224,3)) + 100.0, iter_n=10, step=1.5, octave_n=4, octave_scale=1.4):
	score = tf.reduce_mean(obj)
	gradi = tf.gradients(score, X)[0]

    
 
	img = img_noise
	octaves = []
 
	def tffunc(*argtypes):
		placeholders = list(map(tf.placeholder, argtypes))
		def wrap(f):
			out = f(*placeholders)
			def wrapper(*args, **kw):
				return out.eval(dict(zip(placeholders, args)), session=kw.get('session'))
			return wrapper
		return wrap
	def resize(img, size):
		img = tf.expand_dims(img, 0)
		return tf.image.resize_bilinear(img, size)[0,:,:,:]
 
	resize = tffunc(np.float32, np.int32)(resize)

	for _ in range(octave_n-1):
		hw = img.shape[:2]
		print ('hw:',hw)
		lo = resize(img, np.int32(np.float32(hw)/octave_scale))
		print ('lo:',lo)
		hi = img-resize(lo, hw)
		print ('hi:',hi)
		img = lo
		octaves.append(hi)
 
	def calc_grad_tiled(img, t_grad, tile_size=512):
		sz = tile_size
		h, w = img.shape[:2]
		sx, sy = np.random.randint(sz, size=2)
		img_shift = np.roll(np.roll(img, sx, 1), sy, 0)
		grad = np.zeros_like(img)
		for y in range(0, max(h-sz//2, sz),sz):
			for x in range(0, max(w-sz//2, sz),sz):
				sub = img_shift[y:y+sz,x:x+sz]
				g = sess.run(t_grad, {X:sub})
				grad[y:y+sz,x:x+sz] = g
		return np.roll(np.roll(grad, -sx, 1), -sy, 0)   
 
	for octave in range(octave_n):
		if octave>0:
			hi = octaves[-octave]
			img = resize(img, hi.shape[:2])+hi
		for _ in range(iter_n):
			g = calc_grad_tiled(img, gradi)
			img += g*(step / (np.abs(g).mean()+1e-7))
 
		# 保存图像
		output_file = 'D:/python_code/deep_dream/output' + str(octave+1) + '.jpg'
		cv2.imwrite(output_file, img)
		print(output_file)

 
# 加载输入图像
input_img = cv2.imread('D:/python_code/image/input.jpg')
input_img = np.float32(input_img)
print (input_img.shape)

 
# 选择层
layer = 'mixed4c'
 
deep_dream(tf.square(graph.get_tensor_by_name("import/%s:0"%layer)), input_img)

1
layers: 59
feature: 7548
(640, 1024, 3)
hw: (640, 1024)
lo: [[[ 148.          174.          190.        ]
  [ 146.59918213  172.59918213  188.59918213]
  [ 146.80163574  172.80163574  188.80163574]
  ..., 
  [ 194.20239258  131.20239258   51.        ]
  [ 191.19842529  130.19842529   50.19842529]
  [ 192.59924316  131.59924316   51.59924316]]

 [[ 146.4004364   172.4004364   188.4004364 ]
  [ 144.59918213  170.59918213  186.59918213]
  [ 144.80163574  170.80163574  186.80163574]
  ..., 
  [ 193.80195618  130.80195618   49.80195618]
  [ 191.27787781  130.59886169   49.95690155]
  [ 192.99967957  131.99967957   51.99967957]]

 [[ 147.8008728   173.8008728   189.8008728 ]
  [ 146.20092773  172.20092773  188.20092773]
  [ 145.76138306  171.76138306  187.76138306]
  ..., 
  [ 193.20239258  130.20239258   49.20239258]
  [ 193.32273865  131.35734558   50.07832336]
  [ 193.59924316  133.40011597   51.79836655]]

 ..., 
 [[  30.40270996   95.60406494   73.        ]
  [  24.33149338   90.33149

In [16]:
print (layers)

['import/conv2d0_pre_relu/conv', 'import/conv2d1_pre_relu/conv', 'import/conv2d2_pre_relu/conv', 'import/mixed3a_1x1_pre_relu/conv', 'import/mixed3a_3x3_bottleneck_pre_relu/conv', 'import/mixed3a_3x3_pre_relu/conv', 'import/mixed3a_5x5_bottleneck_pre_relu/conv', 'import/mixed3a_5x5_pre_relu/conv', 'import/mixed3a_pool_reduce_pre_relu/conv', 'import/mixed3b_1x1_pre_relu/conv', 'import/mixed3b_3x3_bottleneck_pre_relu/conv', 'import/mixed3b_3x3_pre_relu/conv', 'import/mixed3b_5x5_bottleneck_pre_relu/conv', 'import/mixed3b_5x5_pre_relu/conv', 'import/mixed3b_pool_reduce_pre_relu/conv', 'import/mixed4a_1x1_pre_relu/conv', 'import/mixed4a_3x3_bottleneck_pre_relu/conv', 'import/mixed4a_3x3_pre_relu/conv', 'import/mixed4a_5x5_bottleneck_pre_relu/conv', 'import/mixed4a_5x5_pre_relu/conv', 'import/mixed4a_pool_reduce_pre_relu/conv', 'import/mixed4b_1x1_pre_relu/conv', 'import/mixed4b_3x3_bottleneck_pre_relu/conv', 'import/mixed4b_3x3_pre_relu/conv', 'import/mixed4b_5x5_bottleneck_pre_relu/conv',

In [17]:
lay=[op.name for op in graph.get_operations()]
len(lay)

676

In [19]:
feature_nums[0:10]

[64, 64, 192, 64, 96, 128, 16, 32, 32, 128]

In [20]:
fea=[graph.get_tensor_by_name(name+':0') for name in layers]

In [21]:
fea[0]

<tf.Tensor 'import/conv2d0_pre_relu/conv:0' shape=(?, ?, ?, 64) dtype=float32>

In [22]:
fea[1]

<tf.Tensor 'import/conv2d1_pre_relu/conv:0' shape=(?, ?, ?, 64) dtype=float32>

In [23]:
fea[2]

<tf.Tensor 'import/conv2d2_pre_relu/conv:0' shape=(?, ?, ?, 192) dtype=float32>

In [24]:
graph.get_tensor_by_name("import/%s:0"%layer)

<tf.Tensor 'import/mixed4c:0' shape=(?, ?, ?, 512) dtype=float32>

In [25]:
tf.square(graph.get_tensor_by_name("import/%s:0"%layer))

<tf.Tensor 'Square_1:0' shape=(?, ?, ?, 512) dtype=float32>

In [26]:
tf.reduce_mean(tf.square(graph.get_tensor_by_name("import/%s:0"%layer)))

<tf.Tensor 'Mean_1:0' shape=() dtype=float32>